In [1]:
%run init_notebookspace.py
from settings import DATA_DIR, MODEL_DIR

DATA_DIR is existant under: C:\Users\LukasGrahl\Documents\GIT\memoire1\data


In [2]:
from config import mod5_params, mod5_shocks

In [3]:
%matplotlib inline

from gEconpy.classes.model import gEconModel
from gEconpy.plotting import plotting as gp

import matplotlib.pyplot as plt
import seaborn as sns
from src.plotting import plot_dfs

import numpy as np
import pandas as pd

import os

load model

In [4]:
mod = gEconModel(os.path.join(MODEL_DIR, 'gEconpy_mod6.gcn'), verbose=True)

Model Building Complete.
Found:
	20 equations
	20 variables
	The following variables were eliminated at user request:
		U_t
	1 stochastic shock
		 1 / 1 have a defined prior. 
	8 parameters
		 1 / 8 have a defined prior. 
	0 calibrating equations
	0 parameters to calibrate
 Model appears well defined and ready to proceed to solving.



In [5]:
# assign parameters
mod.free_param_dict.update(mod5_params)
mod.free_param_dict

{'Theta': 0.75,
 'alpha': 0.35,
 'beta': 0.985,
 'delta': 0.025,
 'psi': 8,
 'rho_A': 0.95,
 'sigma_C': 2,
 'sigma_L': 1.5}

In [6]:
mod.steady_state()
mod.solve_model(solver="gensys")

Steady state found! Sum of squared residuals is 1.0400859941440069e-25
[0, 1, 0]
Norm of deterministic part: 1.035522287
Norm of stochastic part:    1.035522287


fit data

In [ ]:
idata = model.fit(
    fake_data,
    filter_type="univariate",
    draws=10_000,
    n_walkers=36,
    return_inferencedata=True,
    burn_in=1000,
)